In [1]:

import os
import pandas as pd
from PIL import Image
import tf_keras as keras
import numpy as np
import tensorflow as tf


2025-06-04 19:09:36.232843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749053376.324730    4849 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749053376.349186    4849 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749053376.553487    4849 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749053376.553553    4849 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749053376.553559    4849 computation_placer.cc:177] computation placer alr

In [17]:
class ImageDatasetTF:
    def __init__(self, csv_file: str, img_folder: str, image_size=(128, 256), transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_folder = img_folder
        self.image_size = image_size
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]["Image"]
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert('RGB').resize(self.image_size)

        if self.transform:
            image = self.transform(image)
        else:
            image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]

        label = self.data.drop(columns=["Image"]).iloc[idx].values.astype(np.float32)
        return image, label

    def generator(self):
        for i in range(len(self)):
            yield self[i]


In [18]:


# Create datasets
train_dataset = ImageDatasetTF(csv_file='PA-100K/train.csv', img_folder='PA-100K/data')
test_dataset = ImageDatasetTF(csv_file='PA-100K/test.csv', img_folder='PA-100K/data')

sample_img, sample_label = train_dataset[0]
img_shape = sample_img.shape
label_shape = sample_label.shape

train_dataset = tf.data.Dataset.from_generator(
    train_dataset.generator,
    output_signature=(
        tf.TensorSpec(shape=img_shape, dtype=tf.float32),
        tf.TensorSpec(shape=label_shape, dtype=tf.float32)
    )
).batch(64).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    test_dataset.generator,
    output_signature=(
        tf.TensorSpec(shape=img_shape, dtype=tf.float32),
        tf.TensorSpec(shape=label_shape, dtype=tf.float32)
    )
).batch(64).prefetch(tf.data.AUTOTUNE)

In [19]:


input_t=keras.Input(shape=(256,128,3))
efficient_model=keras.applications.efficientnet_v2.EfficientNetV2B3(include_top=False,weights='imagenet',input_tensor=input_t)

In [20]:

for layer in efficient_model.layers[:390]:
  layer.trainable=False
model=keras.models.Sequential()
model.add(efficient_model)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dense(26,activation='sigmoid'))

In [21]:
model.compile(optimizer='adam', loss=keras.losses.BinaryFocalCrossentropy(), metrics=['accuracy'])
model.fit(train_dataset, epochs=12 )

Epoch 1/12


E0000 00:00:1749010266.115578    4658 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/efficientnetv2-b3/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


   1250/Unknown - 355s 275ms/step - loss: 0.3629 - accuracy: 0.4996

2025-06-04 07:16:53.404573: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-06-04 07:16:53.405071: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


1250/1250 [==============================] - 376s 292ms/step - loss: 0.3629 - accuracy: 0.4996 - val_loss: 0.3594 - val_accuracy: 0.5944
Epoch 2/12


2025-06-04 07:17:14.686787: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


1250/1250 [==============================] - 378s 302ms/step - loss: 0.3576 - accuracy: 0.4982 - val_loss: 0.3612 - val_accuracy: 0.5942
Epoch 3/12
 312/1250 [======>.......................] - ETA: 4:57 - loss: 0.3556 - accuracy: 0.4977

KeyboardInterrupt: 

In [ ]:
res_model=keras.applications.ResNet152V2(include_top=False, input_tensor=input_t)
for layer in res_model.layers[:-14]:
    layer.trainable=False
optimizer = keras.optimizers.Adam(learning_rate=3e-3)
model2 = keras.models.Sequential([
    res_model,
    keras.layers.GlobalAveragePooling2D(),  # Critical!
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(26, activation='sigmoid')
   
])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model2.compile(optimizer=optimizer,loss=keras.losses.BinaryCrossentropy(),  metrics=['binary_accuracy', keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()])
model.fit(train_dataset,validation_data= test_dataset, epochs=50)

Epoch 1/50
   1250/Unknown - 315s 252ms/step - loss: 0.3196 - accuracy: 0.4779

In [ ]:
model_checkpoint = keras.callbacks.ModelCheckpoint(
    'best_model.h5', monitor='val_loss', save_best_only=True
)


In [2]:

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))


2.19.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Num GPUs Available: 1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


RuntimeError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [4]:
df= pd.read_csv("PA-100K/train.csv")
print(df.columns)


Index(['Image', 'Female', 'AgeOver60', 'Age18-60', 'AgeLess18', 'Front',
       'Side', 'Back', 'Hat', 'Glasses', 'HandBag', 'ShoulderBag', 'Backpack',
       'HoldObjectsInFront', 'ShortSleeve', 'LongSleeve', 'UpperStride',
       'UpperLogo', 'UpperPlaid', 'UpperSplice', 'LowerStripe', 'LowerPattern',
       'LongCoat', 'Trousers', 'Shorts', 'Skirt&Dress', 'boots'],
      dtype='object')
